In [1]:
from urllib.request import urlopen
from urllib.request import Request 
from urllib.parse import urlencode, quote_plus

import urllib.request as ul
import xmltodict
import json
import sys
import io
import pandas as pd
import requests

### 연월일 분리위한 데이터 프레임 만들기

In [2]:
#start = pd.to_datetime('19881001')
start = pd.to_datetime('20130101')
end = pd.to_datetime('20190331') 
df = pd.DataFrame(columns=['date'])
df['date'] = pd.date_range(start,end,freq='D')

df['Year'] = df['date'].dt.year 
df['Month'] = df['date'].dt.month 
df['Day'] = df['date'].dt.day

df

,date,Year,Month,Day
0,2013-01-01,2013,1,1
1,2013-01-02,2013,1,2
2,2013-01-03,2013,1,3
3,2013-01-04,2013,1,4
4,2013-01-05,2013,1,5
...,...,...,...,...
2276,2019-03-27,2019,3,27
2277,2019-03-28,2019,3,28
2278,2019-03-29,2019,3,29
2279,2019-03-30,2019,3,30


In [3]:
### 월,일의 한자리(ex. 1,2,3...) 앞에 0붙여야함
def convert_month(df):
    for i in range(len(df)):
        month = df['Month'].iloc[i]
        if int(month) < 10:
            month = "0"+str(month)
            df['Month'].iloc[i] = month
            
def convert_day(df):
    for i in range(len(df)):
        day = df['Day'].iloc[i]
        if int(day) < 10:
            day = "0"+str(day)
            df['Day'].iloc[i] = day        

In [4]:
convert_month(df)
convert_day(df)
df

C:\Anaconda3\envs\chaewon\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,date,Year,Month,Day
0,2013-01-01,2013,01,01
1,2013-01-02,2013,01,02
2,2013-01-03,2013,01,03
3,2013-01-04,2013,01,04
4,2013-01-05,2013,01,05
...,...,...,...,...
2276,2019-03-27,2019,03,27
2277,2019-03-28,2019,03,28
2278,2019-03-29,2019,03,29
2279,2019-03-30,2019,03,30


### API로 음력 받아오기

In [5]:
data = pd.DataFrame()


for i in range(len(df)):
    
    Year = df['Year'].iloc[i]
    Month = df['Month'].iloc[i]
    Day = df['Day'].iloc[i]
               
    url = 'http://apis.data.go.kr/B090041/openapi/service/LrsrCldInfoService/getLunCalInfo?serviceKey=MS8w0NUzG28tbYJgzzGzNKYispIbGKI3zFOsb8cV08xuh%2FCqBHlYn%2FfQcLySJxjsN0dvB93Ol%2BVQKftGRQG9ew%3D%3D&solYear='+str(Year)+'&solMonth='+str(Month)+'&solDay='+str(Day)

    requestd = Request(url)
    requestd.get_method = lambda: 'GET'
    response_body = urlopen(requestd).read()
    resp = xmltodict.parse(response_body)
    resp_dic = json.loads(json.dumps(resp))
    #print(resp_dic)
    #print(i)

    data = data.append(resp_dic['response']['body']['items']['item'],ignore_index=True)

print("********end********")

********end********


In [6]:
data

,lunDay,lunIljin,lunLeapmonth,lunMonth,lunNday,lunSecha,lunWolgeon,lunYear,solDay,solJd,solLeapyear,solMonth,solWeek,solYear
0,20,정묘(丁卯),평,11,30,임진(壬辰),임자(壬子),2012,01,2456294,평,01,화,2013
1,21,무진(戊辰),평,11,30,임진(壬辰),임자(壬子),2012,02,2456295,평,01,수,2013
2,22,기사(己巳),평,11,30,임진(壬辰),임자(壬子),2012,03,2456296,평,01,목,2013
3,23,경오(庚午),평,11,30,임진(壬辰),임자(壬子),2012,04,2456297,평,01,금,2013
4,24,신미(辛未),평,11,30,임진(壬辰),임자(壬子),2012,05,2456298,평,01,토,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276,21,계해(癸亥),평,02,29,기해(己亥),정묘(丁卯),2019,27,2458570,평,03,수,2019
2277,22,갑자(甲子),평,02,29,기해(己亥),정묘(丁卯),2019,28,2458571,평,03,목,2019
2278,23,을축(乙丑),평,02,29,기해(己亥),정묘(丁卯),2019,29,2458572,평,03,금,2019
2279,24,병인(丙寅),평,02,29,기해(己亥),정묘(丁卯),2019,30,2458573,평,03,토,2019


In [7]:
raw_api = data[['lunYear','lunMonth','lunDay']]
raw_api

,lunYear,lunMonth,lunDay
0,2012,11,20
1,2012,11,21
2,2012,11,22
3,2012,11,23
4,2012,11,24
...,...,...,...
2276,2019,02,21
2277,2019,02,22
2278,2019,02,23
2279,2019,02,24


### 만들어둔 데이터 프레임이랑 합치기

In [8]:
lun_final = pd.concat([df,raw_api],axis=1) 
lun_final

,date,Year,Month,Day,lunYear,lunMonth,lunDay
0,2013-01-01,2013,01,01,2012,11,20
1,2013-01-02,2013,01,02,2012,11,21
2,2013-01-03,2013,01,03,2012,11,22
3,2013-01-04,2013,01,04,2012,11,23
4,2013-01-05,2013,01,05,2012,11,24
...,...,...,...,...,...,...,...
2276,2019-03-27,2019,03,27,2019,02,21
2277,2019-03-28,2019,03,28,2019,02,22
2278,2019-03-29,2019,03,29,2019,02,23
2279,2019-03-30,2019,03,30,2019,02,24


In [9]:
lun_final.to_csv("./make_data/0B_음력날짜.csv",index=False) ##여기까지 20130101~20190331

In [ ]:
##그 위에 만드는 코드 여기다 작성하기! + 위에 받은거랑 합쳐서 만들기! 

In [ ]:
.to_csv("./make_data/0B_1988_음력날짜.csv",index=False)